## 개념 정리
### 용어 정리
- 앙상블 학습: 더 좋은 예측결과를 만들기 위해, 여러 개의 모델을 학습하는 알고리즘
- 랜덤포레스트
    - 랜덤하게 샘플을 뽑는, 부트스트랩 샘플을 이용하여, 여러 개의 트리를 만듦
- 엑스트라 트리
    - 결정트리를 만들 때 전체 훈련셋을 활용하고, 랜덤하게 트리의 노드를 분할함
    - 성능은 낮지만, 과대적합을 막고 검증셋의 점수를 높이는 효과가 있음
- 그레디언트 부스팅
    - 깊이가 얕은 트리를 연속적으로 추가하여 손실함수를 최소화 함 -> 훈련속도가 느림
    - 학습률 파라미터를 조절하여 훈련속도를 늘릴 수 있으나, 과대적합된 모델을 만들 수 있음
- 히스토그램 기반 그레이디언트 부스팅
    - 훈련구간을 256개로 변환하여 노드의 분할 속도가 빠름.
    - sklearn 뿐만 아니라, XGBoost, LightBGM 같은 라이브러리도 있음
### 핵심 패키지와 함수(sklearn)
- RandomForestClassifier / ExtraTreesClassifier
    - 랜덤포레스트 분류 클래스 / 엑스트라 트리 분류 클래스
    - n_estimators: 앙상블을 구성할 트리의 갯수. 기본값은 100
    - criterion: 불순도를 지정. 'gini', 'entropy' 둘 중 하나 선택
    - max_depth: 트리가 성장할 최대 깊이를 지정. 기본값은 Leaf Node가 순수하거나, min_samples_split보다 샘플 갯수가 적을 때까지 성장
    - min_samples_split: 노드를 나누기 위한 최소 샘플 갯수. 기본값은 2
    - max_features: 최적의 분할을 위해 탐색할 특성의 갯수. 기본값은 특성의 제곱근
    - bootstrap: 부트스트랩 샘플을 사용할 지 지정. 기본값은 True.
    - oob_score: OOB 샘플(부트스트랩이 안된 샘플)을 이용하여 모델을 평가할지 지정. 기본값은 False.
    - n_jobs: 병렬 실행에 사용할 CPU 코어수. 기본값은 1. -1은 모든 코어 사용을 의미
- GradientBoostingClassifier
    - 그레이디언트 부스팅 분류 클래스
    - loss: 손실함수를 지정. 기본값은 로지스틱 손실 함수를 지정하는 'deviance'
    - learning_rate: 트리가 앙상블에 기어하는 정도를 조절. 기본값은 0.1
    - n_estimators: 부스팅 단계를 수행하는 트리의 갯수. 기본값은 100
    - subsample: 사용할 훈련셋의 샘플 비율을 지정. 기본값은 1.0
    - max_depth: 개별 회귀 트리의 최대 깊이. 기본값은 3
- HistGradientBoostingClassifier
    - learning_rate: 학습률 or 감쇠율을 의미. 기본값은 0.1, 1이면 감쇠가 없음
    - max_iter: 부스팅 단계를 수행하는 트리의 갯수. 기본값은 100.
    - max_bins: 입력 데이터를 나눌 구간의 갯수. 기본값은 255(이 보다 큰 값은 지정불가). 1개의 구간이 누락된 값을 위해 추가됨.

In [2]:
'''데이터 불러오기'''
import numpy as np
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine[['class']].to_numpy()

In [3]:
'''데이터 나누기'''
# 훈련셋과 테스트셋 나누기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [5]:
'''랜덤포레스트 학습하기'''
from sklearn.model_selection import cross_validate
from sklearn.ensemble import  RandomForestClassifier
RF = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(RF, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 모델 훈련하고, 특성 중요도 출력
RF.fit(train_input, train_target)
print(RF.feature_importances_)
# OOB(Out of Bug) 점수 출력하기: OOB는 일종의 검증셋 역할을 수행함
RF = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
RF.fit(train_input, train_target)
print(RF.oob_score_)

/Users/gimdong-yeong/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/gimdong-yeong/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/gimdong-yeong/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/gimdong-yeong/opt/anaconda3/lib/python3.9/site-packages/sklearn

0.9973541965122431 0.8905151032797809
[0.23167441 0.50039841 0.26792718]


/var/folders/k5/47cpdm1j3153szhd3zmyj02w0000gn/T/ipykernel_11537/45767820.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RF.fit(train_input, train_target)
/var/folders/k5/47cpdm1j3153szhd3zmyj02w0000gn/T/ipykernel_11537/45767820.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RF.fit(train_input, train_target)


0.8934000384837406


In [14]:
'''엑스트라트리 학습하기'''
from sklearn.ensemble import ExtraTreesClassifier
ET = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(ET, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 모델 훈련하고, 특성 중요도 출력
ET.fit(train_input, train_target)
print(ET.feature_importances_)

/Users/gimdong-yeong/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/gimdong-yeong/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/gimdong-yeong/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/gimdong-yeong/opt/anaconda3/lib/python3.9/site-packages/sklearn

0.9974503966084433 0.8887848893166506
[0.20183568 0.52242907 0.27573525]
{'bootstrap': False, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': -1, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}


/var/folders/k5/47cpdm1j3153szhd3zmyj02w0000gn/T/ipykernel_11537/4095473671.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  ET.fit(train_input, train_target)


In [22]:
'''그레이디언트 부스팅 학습'''
from sklearn.ensemble import GradientBoostingClassifier
GB = GradientBoostingClassifier(random_state=42)
scores = cross_validate(GB, train_input, train_target, n_jobs=-1, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 트리의 갯수를 늘리고, 학습률을 늘려 성능 향상 시키기
GB = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(GB, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
GB.fit(train_input, train_target)
print(GB.feature_importances_)

/Users/gimdong-yeong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/gimdong-yeong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/gimdong-yeong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/gimdong-yeong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array 

0.8881086892152563 0.8720430147331015


/Users/gimdong-yeong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/gimdong-yeong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/gimdong-yeong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/gimdong-yeong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array 

0.9464595437171814 0.8780082549788999


/Users/gimdong-yeong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[0.15872278 0.68010884 0.16116839]


In [26]:
'''히스토그램 기반 그레디언트 부스팅 학습'''
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 히스토그램 기반 그레디언트 부스팅의 특성 중요도 계산
from sklearn.inspection import permutation_importance
hgb.fit(train_input,train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)
# 최종 스코어 확인
hgb.score(test_input, test_target)

/Users/gimdong-yeong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/gimdong-yeong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/gimdong-yeong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/gimdong-yeong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array 

0.9321723946453317 0.8801241948619236


/Users/gimdong-yeong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[0.08876275 0.23438522 0.08027708]
[0.05969231 0.20238462 0.049     ]


0.8723076923076923